# Operation Code 取得用テスト

In [12]:
from lxml import html
import requests
from bs4 import BeautifulSoup

url = "https://www.nesdev.org/obelisk-6502-guide/reference.html#INX"

r = requests.get(url=url)
soup = BeautifulSoup(r.content, "html.parser")
lxml_data = html.fromstring(str(soup))

with open("Reference.html", "w", encoding='utf-8') as f:
    f.write(str(soup))


## OpCode構造
* code: 命令コード
* name: 命令名
* bytes: 命令長
* cycles: 命令サイクル
* addressing_mode: アドレッシングモード

### code

### name

In [37]:
opsHeaders = lxml_data.xpath("//h3/a")
# for opsHead in opsHeaders:
#     print(opsHead.attrib.get('name'))

opsNames = [ opsHead.attrib.get('name') for opsHead in opsHeaders ]
print(opsNames)


['ADC', 'AND', 'ASL', 'BCC', 'BCS', 'BEQ', 'BIT', 'BMI', 'BNE', 'BPL', 'BRK', 'BVC', 'BVS', 'CLC', 'CLD', 'CLI', 'CLV', 'CMP', 'CPX', 'CPY', 'DEC', 'DEX', 'DEY', 'EOR', 'INC', 'INX', 'INY', 'JMP', 'JSR', 'LDA', 'LDX', 'LDY', 'LSR', 'NOP', 'ORA', 'PHA', 'PHP', 'PLA', 'PLP', 'ROL', 'ROR', 'RTI', 'RTS', 'SBC', 'SEC', 'SED', 'SEI', 'STA', 'STX', 'STY', 'TAX', 'TAY', 'TSX', 'TXA', 'TXS', 'TYA']
